<a href="https://colab.research.google.com/github/sarbajeetroy/loan-payback-prediction/blob/main/Predicting_Loan_Payback.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
!pip install google-api-python-client

In [28]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
!ls '/content/drive/My Drive/machine_learning_datasets/predicting_loan_payment/playground-series-s5e11'

sample_submission.csv  test.csv  train.csv


In [30]:
import pandas as pd

data_path = '/content/drive/My Drive/machine_learning_datasets/predicting_loan_payment/playground-series-s5e11/train.csv'

try:
    df_train = pd.read_csv(data_path)
    print('Train data loaded successfully!')
    display(df_train.head())
except FileNotFoundError:
    print(f'Error: The file at {data_path} was not found. Please check the path and try again.')
except Exception as e:
    print(f'An error occurred: {e}')

Train data loaded successfully!


,id,annual_income,debt_to_income_ratio,credit_score,loan_amount,interest_rate,gender,marital_status,education_level,employment_status,loan_purpose,grade_subgrade,loan_paid_back
0,0,29367.99,0.084,736,2528.42,13.67,Female,Single,High School,Self-employed,Other,C3,1.0
1,1,22108.02,0.166,636,4593.10,12.92,Male,Married,Master's,Employed,Debt consolidation,D3,0.0
2,2,49566.20,0.097,694,17005.15,9.76,Male,Single,High School,Employed,Debt consolidation,C5,1.0
3,3,46858.25,0.065,533,4682.48,16.10,Female,Single,High School,Employed,Debt consolidation,F1,1.0
4,4,25496.70,0.053,665,12184.43,10.21,Male,Married,High School,Employed,Other,D1,1.0


In [31]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

from xgboost import XGBClassifier

from sklearn.metrics import accuracy_score


In [32]:
df_train

,id,annual_income,debt_to_income_ratio,credit_score,loan_amount,interest_rate,gender,marital_status,education_level,employment_status,loan_purpose,grade_subgrade,loan_paid_back
0,0,29367.99,0.084,736,2528.42,13.67,Female,Single,High School,Self-employed,Other,C3,1.0
1,1,22108.02,0.166,636,4593.10,12.92,Male,Married,Master's,Employed,Debt consolidation,D3,0.0
2,2,49566.20,0.097,694,17005.15,9.76,Male,Single,High School,Employed,Debt consolidation,C5,1.0
3,3,46858.25,0.065,533,4682.48,16.10,Female,Single,High School,Employed,Debt consolidation,F1,1.0
4,4,25496.70,0.053,665,12184.43,10.21,Male,Married,High School,Employed,Other,D1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
593989,593989,23004.26,0.152,703,20958.37,10.92,Female,Single,High School,Employed,Business,C3,1.0
593990,593990,35289.43,0.105,559,3257.24,14.62,Male,Single,Bachelor's,Employed,Debt consolidation,F5,1.0
593991,593991,47112.64,0.072,675,929.27,14.13,Female,Married,Bachelor's,Employed,Debt consolidation,C1,1.0
593992,593992,76748.44,0.067,740,16290.40,9.87,Male,Single,Bachelor's,Employed,Debt consolidation,B2,1.0


In [33]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 593994 entries, 0 to 593993
Data columns (total 13 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    593994 non-null  int64  
 1   annual_income         593994 non-null  float64
 2   debt_to_income_ratio  593994 non-null  float64
 3   credit_score          593994 non-null  int64  
 4   loan_amount           593994 non-null  float64
 5   interest_rate         593994 non-null  float64
 6   gender                593994 non-null  object 
 7   marital_status        593994 non-null  object 
 8   education_level       593994 non-null  object 
 9   employment_status     593994 non-null  object 
 10  loan_purpose          593994 non-null  object 
 11  grade_subgrade        593994 non-null  object 
 12  loan_paid_back        593994 non-null  float64
dtypes: float64(5), int64(2), object(6)
memory usage: 58.9+ MB


In [34]:
# Preprocessing

In [35]:
{column: len(df_train[column].unique()) for column in df_train.columns}

{'id': 593994,
 'annual_income': 119728,
 'debt_to_income_ratio': 526,
 'credit_score': 399,
 'loan_amount': 111570,
 'interest_rate': 1454,
 'gender': 3,
 'marital_status': 4,
 'education_level': 5,
 'employment_status': 5,
 'loan_purpose': 8,
 'grade_subgrade': 30,
 'loan_paid_back': 2}

In [36]:
def preprocess_inputs(df):
  df = df.copy()

  # Drop ID column
  df = df.drop('id', axis=1)
  y = df['loan_paid_back']
  X = df.drop('loan_paid_back', axis=1)

  return X,y

In [37]:
# Get full X and y
X, y = preprocess_inputs(df_train)
print("Features (X) DataFrame head:")
display(X.head())
print("Target (y) Series head:")
display(y.head())

Features (X) DataFrame head:


,annual_income,debt_to_income_ratio,credit_score,loan_amount,interest_rate,gender,marital_status,education_level,employment_status,loan_purpose,grade_subgrade
0,29367.99,0.084,736,2528.42,13.67,Female,Single,High School,Self-employed,Other,C3
1,22108.02,0.166,636,4593.10,12.92,Male,Married,Master's,Employed,Debt consolidation,D3
2,49566.20,0.097,694,17005.15,9.76,Male,Single,High School,Employed,Debt consolidation,C5
3,46858.25,0.065,533,4682.48,16.10,Female,Single,High School,Employed,Debt consolidation,F1
4,25496.70,0.053,665,12184.43,10.21,Male,Married,High School,Employed,Other,D1


Target (y) Series head:


,loan_paid_back
0,1.0
1,0.0
2,1.0
3,1.0
4,1.0


### Data Range and Types

In [38]:
print('Column Data Types and Value Ranges:')
for column in df_train.columns:
    if df_train[column].dtype in ['int64', 'float64']:
        print(f"- {column} ({df_train[column].dtype}): Min = {df_train[column].min()}, Max = {df_train[column].max()}")
    else: # Object type (categorical)
        unique_count = df_train[column].nunique()
        if unique_count <= 10: # Display unique values if there aren't too many
            print(f"- {column} ({df_train[column].dtype}): Unique values = {df_train[column].unique()}")
        else:
            print(f"- {column} ({df_train[column].dtype}): Number of unique values = {unique_count}")

Column Data Types and Value Ranges:
- id (int64): Min = 0, Max = 593993
- annual_income (float64): Min = 6002.43, Max = 393381.74
- debt_to_income_ratio (float64): Min = 0.011, Max = 0.627
- credit_score (int64): Min = 395, Max = 849
- loan_amount (float64): Min = 500.09, Max = 48959.95
- interest_rate (float64): Min = 3.2, Max = 20.99
- gender (object): Unique values = ['Female' 'Male' 'Other']
- marital_status (object): Unique values = ['Single' 'Married' 'Divorced' 'Widowed']
- education_level (object): Unique values = ['High School' "Master's" "Bachelor's" 'PhD' 'Other']
- employment_status (object): Unique values = ['Self-employed' 'Employed' 'Unemployed' 'Retired' 'Student']
- loan_purpose (object): Unique values = ['Other' 'Debt consolidation' 'Home' 'Education' 'Vacation' 'Car'
 'Medical' 'Business']
- grade_subgrade (object): Number of unique values = 30
- loan_paid_back (float64): Min = 0.0, Max = 1.0


In [39]:
{column: len(X[column].unique()) for column in X.columns}

{'annual_income': 119728,
 'debt_to_income_ratio': 526,
 'credit_score': 399,
 'loan_amount': 111570,
 'interest_rate': 1454,
 'gender': 3,
 'marital_status': 4,
 'education_level': 5,
 'employment_status': 5,
 'loan_purpose': 8,
 'grade_subgrade': 30}

In [40]:
{column: list(X[column].unique()) for column in X.select_dtypes('object').columns}

{'gender': ['Female', 'Male', 'Other'],
 'marital_status': ['Single', 'Married', 'Divorced', 'Widowed'],
 'education_level': ['High School', "Master's", "Bachelor's", 'PhD', 'Other'],
 'employment_status': ['Self-employed',
  'Employed',
  'Unemployed',
  'Retired',
  'Student'],
 'loan_purpose': ['Other',
  'Debt consolidation',
  'Home',
  'Education',
  'Vacation',
  'Car',
  'Medical',
  'Business'],
 'grade_subgrade': ['C3',
  'D3',
  'C5',
  'F1',
  'D1',
  'D5',
  'C2',
  'C1',
  'F5',
  'D4',
  'C4',
  'D2',
  'E5',
  'B1',
  'B2',
  'F4',
  'A4',
  'E1',
  'F2',
  'B4',
  'E4',
  'B3',
  'E3',
  'B5',
  'E2',
  'F3',
  'A5',
  'A3',
  'A1',
  'A2']}

In [41]:
# ---- Split ----
X_train_raw, X_test_raw, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

In [ ]:
!pip install category_encoders

In [ ]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from category_encoders import HashingEncoder

# ---- Encoding + scaling setup ----
one_hot_cols = ['gender', 'marital_status', 'employment_status', 'loan_purpose']
ordinal_cols = ['education_level']
hashing_cols = ['grade_subgrade']
categorical_cols = one_hot_cols + ordinal_cols + hashing_cols
numeric_cols = [c for c in X.columns if c not in categorical_cols]

one_hot_encoder = OneHotEncoder(sparse_output=False, drop='first')
one_hot_encoder.fit(X_train_raw[one_hot_cols])

education_order = ['High School', "Other", "Bachelor's", "Master's", 'PhD']
ordinal_encoder = OrdinalEncoder(categories=[education_order])
ordinal_encoder.fit(X_train_raw[ordinal_cols])

hashing_encoder = HashingEncoder(cols=hashing_cols, n_components=8, drop_invariant=True)
hashing_encoder.fit(X_train_raw[hashing_cols])

scaler = StandardScaler()
scaler.fit(X_train_raw[numeric_cols])

def transform_full(X_raw):
    one_hot = one_hot_encoder.transform(X_raw[one_hot_cols])
    one_hot_df = pd.DataFrame(
        one_hot,
        columns=one_hot_encoder.get_feature_names_out(one_hot_cols),
        index=X_raw.index
    )

    ordinal = ordinal_encoder.transform(X_raw[ordinal_cols])
    ordinal_df = pd.DataFrame(ordinal, columns=ordinal_cols, index=X_raw.index)

    hashing = hashing_encoder.transform(X_raw[hashing_cols])
    hashing_df = hashing_encoder.transform(X_raw[hashing_cols])
    hashing_df.index = X_raw.index  # ensure index alignment

    X_num = X_raw[numeric_cols].copy()
    X_num[numeric_cols] = scaler.transform(X_num[numeric_cols])

    X_proc = pd.concat([X_num, one_hot_df, ordinal_df, hashing_df], axis=1)
    return X_proc

X_train = transform_full(X_train_raw)
X_test = transform_full(X_test_raw)

In [ ]:
from sklearn.preprocessing import StandardScaler

# Identify numerical columns for scaling
numerical_cols = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Initialize StandardScaler
scaler = StandardScaler()

# Apply scaling to numerical columns
X_scaled = X_train.copy()
X_scaled[numerical_cols] = scaler.fit_transform(X_scaled[numerical_cols])

print('Scaled X DataFrame head:')
display(X_scaled.head())
print('\nScaled X DataFrame info:')
X_scaled.info()

In [ ]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

# Identify columns for one-hot encoding
one_hot_cols = ['gender', 'marital_status', 'employment_status', 'loan_purpose']

# Apply one-hot encoding
one_hot_encoder = OneHotEncoder(sparse_output=False, drop='first') # drop='first' to avoid multicollinearity
one_hot_encoded_features = one_hot_encoder.fit_transform(X[one_hot_cols])
one_hot_df = pd.DataFrame(one_hot_encoded_features, columns=one_hot_encoder.get_feature_names_out(one_hot_cols), index=X.index)

# Identify column for ordinal encoding and define the order
education_order = ['High School', "Other", "Bachelor's", "Master's", 'PhD']
ordinal_cols = ['education_level']

# Apply ordinal encoding
ordinal_encoder = OrdinalEncoder(categories=[education_order])
ordinal_encoded_features = ordinal_encoder.fit_transform(X[ordinal_cols])
ordinal_df = pd.DataFrame(ordinal_encoded_features, columns=ordinal_cols, index=X.index)

# Drop original categorical columns from X and concatenate encoded features
X_processed = X.drop(columns=one_hot_cols + ordinal_cols + ['grade_subgrade']).copy()
X_processed = pd.concat([X_processed, one_hot_df, ordinal_df], axis=1)

In [ ]:
print('Processed X DataFrame head:')
display(X_processed.head())
print('\nProcessed X DataFrame info:')
X_processed.info()

# Task
Initialize and train Logistic Regression, Decision Tree, Random Forest, Support Vector Machine, K-Nearest Neighbors, Gaussian Naive Bayes, and XGBoost classifiers on the preprocessed training data (X_train, y_train), then evaluate and summarize the accuracy scores for each model using the test data (X_test, y_test).

## Initialize Models

### Subtask:
Create instances of LogisticRegression, DecisionTreeClassifier, RandomForestClassifier, SVC, KNeighborsClassifier, GaussianNB, and XGBClassifier and store them in a dictionary.


**Reasoning**:
To initialize the machine learning models, I will create an empty dictionary and then instantiate each specified model with its respective parameters, adding each to the dictionary.



In [ ]:
models = {
    'Logistic Regression': LogisticRegression(solver='liblinear', random_state=42),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42),
    'Support Vector Machine': SVC(random_state=42),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Gaussian Naive Bayes': GaussianNB(),
    'XGBoost': XGBClassifier(random_state=42)
}

print('Models initialized successfully!')
for name, model in models.items():
    print(f'- {name}: {model}')

## Train Models

### Subtask:
Iterate through the dictionary of initialized models, training each model on the `X_train` and `y_train` datasets. Store the trained models back into the `models` dictionary.

**Reasoning**:
Now that the models are initialized, I will iterate through the `models` dictionary and train each model using the `X_train` and `y_train` datasets.



In [ ]:
for name, model in models.items():
    print(f'Training {name}...')
    model.fit(X_train, y_train)
    print(f'{name} trained successfully!')

print('\nAll models trained successfully!')